In [45]:
import os, pandas as pd, geopandas as gpd, requests, folium
from bs4 import BeautifulSoup

In [47]:
import requests
from bs4 import BeautifulSoup

# URL for the feature layer's metadata
url = "https://services.arcgis.com/S9th0jAJ7bqgIRjw/ArcGIS/rest/services/Major_Crime_Indicators_Open_Data/FeatureServer/0"


# Fetch the page
response = requests.get(url)

# Ensure the request was successful
if response.status_code == 200:
    # Use response.text as the input for BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <b> tags, then filter by those containing 'Last Edit Date:'
    b_tags = soup.find_all('b')
    last_edit_date = None
    for tag in b_tags:
        if 'Last Edit Date:' in tag.text:
            # The date should be the next sibling of the <b> tag
            last_edit_date = tag.next_sibling.strip()

    if last_edit_date:
        print(f"Last Edit Date: {last_edit_date}")
    else:
        print("Last Edit Date not found.")
else:  
    print("Failed to fetch the webpage.")

#print(last_edit_date)


# Write to a text file
# with open('crime/last_edit_date.txt', 'w') as file:
#     file.write(last_edit_date)
#     print('Lsst edit date written to ../crime/last_edit_date.txt')

Last Edit Date: 1/10/2024 8:52:27 PM


In [51]:
# Path to the file where the last edit date is stored
file_path = "crime/last_edit_date.txt"

# Function to read the last stored edit date from file
def read_last_stored_date(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read().strip()
            
    except FileNotFoundError:
        return None

# Read the last stored edit date from the file
stored_last_edit_date = read_last_stored_date(file_path)
print(stored_last_edit_date)

1/10/2024 8:52:27 PM


In [53]:
# Compare the current last edit date to the stored one
if last_edit_date != stored_last_edit_date:
    print("The dataset has been updated. Proceeding with data scraping...")
else:
    print("No updates.")


No updates.


In [55]:
### Get IDs to batch request
import requests

# Base URL for the feature service
base_url = "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Major_Crime_Indicators_Open_Data/FeatureServer/0/query"

# Parameters for fetching all object IDs with OCC_YEAR > 2020
params_ids = {
    'where': '1=1',
    'returnIdsOnly': 'true',
    'f': 'json'
}

# Make the request for IDs
response_ids = requests.get(base_url, params=params_ids)
object_ids = response_ids.json().get('objectIds', []) if response_ids.status_code == 200 else []

print(f"Total IDs fetched: {len(object_ids)}")
api_data_count = len(object_ids)
print(api_data_count)

Total IDs fetched: 372899
372899


In [62]:
# Load existing data and count records
existing_data = gpd.read_file("crime/data/Major_Crime_Indicators_Open_Data.geojson")
existing_records_count = len(existing_data)
print(existing_records_count)

KeyboardInterrupt: 

/Users/tomweatherburn/Library/CloudStorage/OneDrive-Personal/dev/tdubolyou.github.io


In [57]:

# Calculate the number of new records to fetch
records_to_fetch = api_data_count - existing_records_count
print(records_to_fetch)

NameError: name 'existing_records_count' is not defined

In [40]:
import requests
import geopandas as gpd
import pandas as pd

def fetch_data_in_batches(object_ids, batch_size=200):
    """Fetches data in batches and returns a GeoDataFrame of the combined results."""
    gdfs = []  # List to store GeoDataFrames for each batch
    for i in range(0, len(object_ids), batch_size):
        batch_ids = object_ids[i:i + batch_size]
        # Convert the list of IDs into a string format accepted by the API
        ids_str = ','.join(map(str, batch_ids))
        
        # Prepare the request URL and parameters for the batch
        batch_params = {
            'where': f'OBJECTID IN ({ids_str})',
            'outFields': '*',
            'f': 'geojson'
        }
        
        response = requests.get(base_url, params=batch_params)
        
        if response.status_code == 200:
            data = response.json()
            batch_gdf = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')
            gdfs.append(batch_gdf)
            print("Appended records ending in "+ str(batch_ids[-1]))
        else:
            print(f"Error fetching batch: {i // batch_size + 1}, Status Code: {response.status_code}")
            break  # Stop fetching more batches after the first error

    if gdfs:
        # Combine all batch GeoDataFrames into one
        full_gdf = pd.concat(gdfs, ignore_index=True)
        return full_gdf
    else:
        return gpd.GeoDataFrame()

# Assuming `object_ids` contains the list of IDs to fetch
full_gdf = fetch_data_in_batches(object_ids, batch_size=200)

if not full_gdf.empty:
    print(f"Successfully fetched data for {len(full_gdf)} features.")
else:
    print("Failed to fetch data.")


Appended records ending in 200
Appended records ending in 400
Appended records ending in 600
Appended records ending in 800
Appended records ending in 1000
Appended records ending in 1200
Appended records ending in 1400
Appended records ending in 1600
Appended records ending in 1800
Appended records ending in 2000
Appended records ending in 2200
Appended records ending in 2400
Appended records ending in 2600
Appended records ending in 2800
Appended records ending in 3000
Appended records ending in 3200
Appended records ending in 3400
Appended records ending in 3600
Appended records ending in 3800
Appended records ending in 4000
Appended records ending in 4200
Appended records ending in 4400
Appended records ending in 4600
Appended records ending in 4800
Appended records ending in 5000
Appended records ending in 5200
Appended records ending in 5400
Appended records ending in 5600
Appended records ending in 5800
Appended records ending in 6000
Appended records ending in 6200
Appended rec

KeyboardInterrupt: 